In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#dos librerias nuevas que usamos en APIs
import urllib
import json



# GCBA api

In [ ]:
#una forma elemental de consultar una api es usando el navegador
'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [ ]:
url = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [ ]:
#esto se descompone en varios elementos
url_base = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?'
client_id = 'client_id=02f9fb1291ef41f6bfa93b0d99779291'
client_secret = 'client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [ ]:
#cada API puede ser diferente y debemos chequear su documentación, pero los diferentes parametros
#se separan en general con &
url_computesta = url_base + client_id + '&' + client_secret
url_computesta

In [ ]:
url_computesta == url

En este caso los unicos dos parametros son `client_id` y `client_secret`. Normalmente las APIs piden algun tipo de _token_ o clave secreta **que debemos evitar compartir (como estoy haciendo yo aca)**. En los casos siguientes vamos a ver otros ejemplos con consultas mas complejas

In [ ]:
#realizamos el request a esa url
response = urllib.request.urlopen(url)
response

In [ ]:
#la libreria json nos permite leer esta informacion como un diccionario de Python
datos = json.loads(response.read())


In [ ]:
#inspeccionemos nuestros datos
type(datos)

In [ ]:
datos['data']['stations'][0]

In [ ]:
pd.DataFrame({'nombres':['felipe','diego'],
             'edades':[15,22]})

In [ ]:
pd.DataFrame(datos['data']['stations'])

In [ ]:
#dado el formato en el que se 
estaciones = pd.DataFrame(datos['data']['stations'])
estaciones = estaciones.reindex(columns=['station_id', 'name', 'capacity', 'lat', 'lon'])
estaciones.head()

In [ ]:
url_base = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?'
url_computesta = url_base + client_id + '&' + client_secret
url_computesta

In [ ]:
#conusltamos el status de las estaciones
response = urllib.request.urlopen(url_computesta)
status = json.loads(response.read())
status = pd.DataFrame(status['data']['stations'])


In [ ]:
status

In [ ]:
datetime.now()

In [ ]:
#extraemos el momento en que consultamos los datos, especificando que nuestro timezone es el de Argentina
from datetime import datetime
import pytz

argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
tiempo_consulta = datetime.now(tz=argentina_tz)

#conusltamos el status de las estaciones
response = urllib.request.urlopen(url_computesta)
status = json.loads(response.read())
status = pd.DataFrame(status['data']['stations'])

#mergeamos ambos datasets para tener para cada status, la informacion de la estacion
status = status.merge(estaciones, how='inner', on='station_id')
status['timestamp_consulta'] = tiempo_consulta
status.to_csv('../data/' + str(tiempo_consulta)[:16] + '.csv', index=False)


In [ ]:
status.head()

Aca pueden ver una funcion que hace esto y se se deja corriendo consulta y guarda cada media hora

In [ ]:
#https://github.com/alephcero/ecobici_un_dia
import os
import urllib.request as urllib
import json
import pandas as pd
from datetime import datetime
from time import sleep
import pytz


argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')


def consultar_y_persistir():
    url_datos = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=' + \
        os.environ['ECOBICIID'] + '&client_secret=' + \
        os.environ['ECOBICISECRET']
    response = urllib.urlopen(url_datos)
    datos = json.loads(response.read())
    estaciones = pd.DataFrame(datos['data']['stations'])
    estaciones = estaciones.reindex(
        columns=['station_id', 'name', 'capacity', 'lat', 'lon'])

    url_consulta = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?client_id=' + \
        os.environ['ECOBICIID'] + '&client_secret=' + \
        os.environ['ECOBICISECRET']
    argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
    tiempo_consulta = datetime.now(tz=argentina_tz)
    response = urllib.urlopen(url_consulta)
    status = json.loads(response.read())
    dt = pd.DataFrame(status['data']['stations'])
    dt = dt.merge(estaciones, how='inner', on='station_id')
    dt['timestamp_consulta'] = tiempo_consulta
    dt.to_csv('data/' + str(tiempo_consulta)[:16] + '.csv', index=False)


no_es_medianoche = True
while no_es_medianoche:
    if (datetime.now(tz=argentina_tz).hour == 0) and (datetime.now(tz=argentina_tz).minute == 0):
        no_es_medianoche = False
    print("waiting", datetime.now(tz=argentina_tz))
    sleep(60 * 1)

for i in range(48):
    sleep(60 * 30)
    print(i)
    consultar_y_persistir()

# Google

In [3]:
#Primero creamos un dataset para testear
#formato google maps latlong, yx
obelisco = (-34.60377,-58.38161)
retiro = (-34.591372, -58.374261)
once = (-34.608315, -58.406169)
constitucion = (-34.627554, -58.380690)

def construir_dict(nombre,tupla_latlong):
    return {
        'nombre':nombre,
        'coordenadas':{
            'long':tupla_latlong[1],
            'lat':tupla_latlong[0],
            'x':tupla_latlong[1],
            'y':tupla_latlong[0]
        }
}
    
puntos = [construir_dict('obelisco',(-34.60377,-58.38161)),
         construir_dict('retiro',(-34.591372, -58.374261)),
         construir_dict('once',(-34.608315, -58.406169)),
         construir_dict('constitucion',(-34.627554, -58.380690))]
crs = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
geoserie = gpd.GeoSeries([Point(punto['coordenadas']['x'],punto['coordenadas']['y']) for punto in puntos],crs=crs) 
geodt = gpd.GeoDataFrame({'geometry': geoserie, 'nombre':[punto['nombre'] for punto in puntos]})
geodt.index=geodt.nombre
geodt

,geometry,nombre
nombre,,
obelisco,POINT (-58.38161 -34.60377),obelisco
retiro,POINT (-58.37426 -34.59137),retiro
once,POINT (-58.40617 -34.60831),once
constitucion,POINT (-58.38069 -34.62755),constitucion


In [4]:
# veamos la documentacion para obtener un token 
# https://developers.google.com/maps/documentation/distance-matrix/get-api-key
# a esta documentacion le falta el paso de habilitar ENABLE
api_token = ''

Nuestra intencion es obtener tiempos de viaje por eso usaremos Distance Matrix API. Para ello podemos leer la [documentacion](https://developers.google.com/maps/documentation/distance-matrix/intro#DistanceMatrixRequests).


In [ ]:
#creamos nuestras coordenadas en el formato requerido por la API de google
origen_lat = geodt.loc['obelisco','geometry'].y
origen_long = geodt.loc['obelisco','geometry'].x
destino_lat = geodt.loc['retiro','geometry'].y
destino_long = geodt.loc['retiro','geometry'].x

In [ ]:
origen_lat

In [ ]:
#construimos la consulta con el mismo metodo que vimos antes
url_base = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
units = 'units=metric' 
origins = 'origins=' + str(origen_lat) + ',' + str(origen_long)
destinations = 'destinations=%f,%f' % (destino_lat,destino_long)
key = 'key={0}'.format(api_token)

url = url_base + '&' + units + '&' + origins + '&' + destinations + '&' + key
url

In [ ]:
#volvemos a realizar la consulta y leerla como diccionario de python
response = urllib.request.urlopen(url)
diccio = json.loads(response.read())
diccio

In [ ]:
#podemos agregar un nuevo parametro, el modo de transporte y agregarlo a lo que ya veniamos consultando
mode = 'mode=bicycling'

url = url_base + '&' + units + '&' + origins + '&' + destinations + '&'+ mode + '&' + key
url

In [ ]:
#repetimos el proceso de consulta
response = urllib.request.urlopen(url)
diccio_bici = json.loads(response.read())
diccio_bici

In [5]:
#afortunadamente la gente de google creo una libreria que hace todo esto por nosotros
#https://github.com/googlemaps/google-maps-services-python
import googlemaps

In [6]:
#documentacion
#https://github.com/googlemaps/google-maps-services-python
gmaps = googlemaps.Client(key=api_token)

In [7]:
#realizamos una consulta con la liberia
travel = gmaps.distance_matrix(
    origins = once,
    destinations = obelisco,
    mode='transit', traffic_model='best_guess',
    transit_routing_preference = 'fewer_transfers')

In [8]:
#vemos el resultado de nuestro consulta
travel

{'destination_addresses': ['Av. Corrientes 1031, C1010 CABA, Argentina'],
 'origin_addresses': ['Av. Pueyrredón 143, C1032ABB CABA, Argentina'],
 'rows': [{'elements': [{'distance': {'text': '2.8 km', 'value': 2762},
     'duration': {'text': '13 mins', 'value': 762},
     'fare': {'currency': 'ARS', 'text': 'ARS\xa019.00', 'value': 19},
     'status': 'OK'}]}],
 'status': 'OK'}

In [9]:
#vemos que es un diccionario
type(travel)

dict

In [10]:
#podemos convertirlo en un pandas data frame a partir de la estructura de diccionario
data = pd.DataFrame(travel)
data

,destination_addresses,origin_addresses,rows,status
0,"Av. Corrientes 1031, C1010 CABA, Argentina","Av. Pueyrredón 143, C1032ABB CABA, Argentina","{'elements': [{'distance': {'text': '2.8 km', ...",OK


In [11]:
#como extraer la informacion que deseamos?
travel['rows'][0]['elements'][0]['duration']['value'] / 60

12.7

In [12]:
#modulizamos esta accion en una funcion
def extraer_duracion(consulta):
    return consulta['rows'][0]['elements'][0]['duration']['value'] / 60

In [13]:
extraer_duracion(travel)

12.7

In [15]:
#creamos una copia de nuestor dataset de prueba y vamos a fijar el destino
geodt_destino = geodt.copy()

In [16]:
destino = [geodt_destino.loc['obelisco','geometry'].y,geodt_destino.loc['obelisco','geometry'].x]
destino

[-34.60377, -58.38161]

In [17]:
#modularizamos nuestras tareas en dos funciones
def extraer_duracion(consulta):
    return consulta['rows'][0]['elements'][0]['duration']['value'] / 60

def consultar_od(orig_x,orig_y,dest_x,dest_y):
    #consulto a API google maps el origen y destino 
    travel = gmaps.distance_matrix(
        origins = (orig_y,orig_x),
        destinations = (dest_y,dest_x),
        mode='transit', traffic_model='best_guess',
        transit_routing_preference = 'fewer_transfers')
    #extraigo la duracion
    output = extraer_duracion(travel)
    return output

In [18]:
#inicializamos una lista vacia donde iremos guardando los tiempos de viaje al obelisco
tiempos = []
for i in geodt.index:
    #en caso de que coincida origen y destino, le imputamos como tiempo 0 minutos
    if i == 'obelisco':
        tiempo = 0
    else:
    #caso contario consultamos a google y guardamos el resultado en tiempo
        tiempo = consultar_od(orig_x = geodt_destino.loc[i,'geometry'].x,
                         orig_y = geodt_destino.loc[i,'geometry'].y,
                         dest_x = geodt_destino.loc['obelisco','geometry'].x,
                         dest_y = geodt_destino.loc['obelisco','geometry'].y)
    #guardamos los resultados en la lista
    tiempos.append(tiempo)
#creamos una columna a partir de la lista
geodt_destino['tiempos_al_obelisco'] = tiempos

In [19]:
#vemos el resultado
geodt_destino

,geometry,nombre,tiempos_al_obelisco
nombre,,,
obelisco,POINT (-58.38161 -34.60377),obelisco,0.000000
retiro,POINT (-58.37426 -34.59137),retiro,11.316667
once,POINT (-58.40617 -34.60831),once,12.700000
constitucion,POINT (-58.38069 -34.62755),constitucion,14.616667


In [20]:
#volvemos a realizar la misma tarea, solamente que hacemos un loop doble,
#para cada destino
#y dentro de cada destino, para cada origen
for destino in geodt.index:
    tiempos = []
    for origen in geodt.index:
        if origen == destino:
            tiempo = 0
        else:
            tiempo = consultar_od(orig_x = geodt.loc[origen,'geometry'].x,
                         orig_y = geodt.loc[origen,'geometry'].y,
                         dest_x = geodt.loc[destino,'geometry'].x,
                         dest_y = geodt.loc[destino,'geometry'].y)
        tiempos.append(tiempo)
    geodt['tiempos_al_' + destino] = tiempos

In [21]:
#observamos el resultado
geodt

,geometry,nombre,tiempos_al_obelisco,tiempos_al_retiro,tiempos_al_once,tiempos_al_constitucion
nombre,,,,,,
obelisco,POINT (-58.38161 -34.60377),obelisco,0.000000,10.133333,13.366667,14.033333
retiro,POINT (-58.37426 -34.59137),retiro,11.316667,0.000000,38.733333,17.166667
once,POINT (-58.40617 -34.60831),once,12.700000,18.366667,0.000000,32.616667
constitucion,POINT (-58.38069 -34.62755),constitucion,14.616667,17.166667,38.066667,0.000000
